In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from tqdm import tqdm
from scipy.fftpack import fft
from matplotlib.pylab import mpl
import csv


%matplotlib qt5
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
mpl.rcParams['axes.unicode_minus'] = False  #显示负号
os.getcwd()

In [7]:
def cal_linear(energy, inter, mid, interval_num, idx = 0):
    # 初始化横坐标
    x = np.array([])
    for i in inter:
        if i != 0:
            x = np.append(x, np.linspace(i, i * 10, interval_num, endpoint=False))
        else:
            x = np.append(x, np.linspace(i, 1, interval_num, endpoint=False))
    
    # 初始化纵坐标
    y = np.zeros(x.shape[0])
    for i in energy:
        while True:
            try:
                if x[idx] <= i < x[idx + 1]:
                    y[idx] += 1
                    break
            except IndexError:
                if x[idx] <= i:
                    y[idx] += 1
                    break
            idx += 1
    
    # 对横坐标作进一步筛选，计算概率分布值
    x, y = x[y != 0], y[y != 0]
    xx = np.zeros(x.shape[0])
    yy = y / sum(y)
    
    # 取区间终点作为该段的横坐标
    for idx in range(len(x) - 1):
        xx[idx] = (x[idx] + x[idx + 1]) / 2
    xx[-1] = x[-1]
    
    # 计算分段区间长度，从而求得概率密度值
    interval = []
    for i, j in enumerate(mid):
        try:
            num = len(np.intersect1d(np.where(inter[i] <= xx)[0], 
                                     np.where(xx < inter[i + 1])[0]))
            interval.extend([j] * num)
        except IndexError:
            num = len(np.where(inter[i] <= xx)[0])
            interval.extend([j] * num)
    yy = yy / np.array(interval)
# #     取对数变换为线性关系
#     log_xx = np.log10(xx)
#     log_yy = np.log10(yy)
#     fit = np.polyfit(log_xx, log_yy, 1)
#     alpha = abs(fit[0])
#     fit_x = np.linspace(min(log_xx), max(log_xx), 100)
#     fit_y = np.polyval(fit, fit_x)
    return xx, yy


def cal_PDF(tmp, inter, mid, interval_num, features_path):
    xx, yy = cal_linear(sorted(tmp), inter, mid, interval_num)
    with open(features_path[:-4] + ' ' + ylabel + '.txt', 'w') as f:
        f.write('{}, {}\n'.format(xlabel, ylabel))
        for j in range(xx.shape[0]):
            f.write('{}, {}\n'.format(xx[j], yy[j]))
    

def cal_ML(tmp, N, features_path):
    ML_y = []
    Error_bar = []
    for j in tqdm(range(N)):
        valid_x = sorted(tmp)[j:]
        E0 = valid_x[0]
        Sum = np.sum(np.log(valid_x/E0))
        N_prime = N - j
        alpha = 1 + N_prime / Sum
        error_bar = (alpha - 1) / pow(N_prime, 0.5)
        ML_y.append(alpha)
        Error_bar.append(error_bar)
    
    with open(features_path[:-4] + ' ' + xlabel[:-4] + '_ML.txt', 'w') as f:
        f.write('{}, ε, Error bar\n'.format(xlabel))
        for j in range(len(ML_y)):
            f.write('{}, {}, {}\n'.format(sorted(tmp)[j], ML_y[j], Error_bar[j]))

In [ ]:
path =  r'C:/Users/Yuan/Desktop'
os.chdir(path)
csv_file = r'./data.csv'
data = pd.read_csv(csv_file)
N = data.shape[0]

# Initializes the number of section
interval_num = 6
interval = 1 / interval_num
interz = []
midz = []

xlabelz = ['Amplitude(μV)', 'Duration(μs)', 'Energy(aJ)']
ylabelz = ['PDF(A)', 'PDF(D)', 'PDF(E)']
feature_idx = [1, 2, 3] # Corresponds to an index in a CSV file

# Generates intervals based on data and number of section
for idx in feature_idx:
    tmp = data.iloc[:, idx]
    tmp_max = int(max(tmp))
    tmp_min = int(min(tmp))
    if tmp_min <= 0:
        interz.append([0] + [pow(10, i) for i in range(len(str(tmp_max)))])
        midz.append([interval * pow(10, i)
                     for i in range(len(str(tmp_max)) + 1)])
    else:
        interz.append([pow(10, i) for i in range(len(str(tmp_min)) - 1, 
                                                 len(str(tmp_max)))])
        midz.append([interval * pow(10, i) 
                     for i in range(len(str(tmp_min)), 
                                    len(str(tmp_max)) + 1)])

for i, [idx, inter, mid, xlabel, ylabel] in enumerate(zip(
    feature_idx, interz, midz, xlabelz, ylabelz)):
    tmp = np.array(data.iloc[:, idx])
    cal_PDF(tmp, inter, mid, interval_num, csv_file)
    cal_ML(tmp, N, csv_file)